In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/detectors/detector_resnext101.py')

root='../../dataset/'

epoch = 'epoch_20'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
#cfg.data.test.pipeline[1]['img_scale'] = (1024,1024) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 10

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/atss_swinl_fpn_dyhead_1x_coco'

#cfg.model.roi_head.bbox_head.num_classes = 10
#cfg.model.roi_head.bbox_head[0].num_classes = 10
#cfg.model.roi_head.bbox_head[1].num_classes = 10
#cfg.model.roi_head.bbox_head[2].num_classes = 10


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [3]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [24]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
#model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/atss_swinl_fpn_dyhead_1x_coco/epoch_20.pth
The model and loaded state dict do not match exactly

missing keys in source state_dict: bbox_head.cls_convs.0.conv.weight, bbox_head.cls_convs.0.gn.weight, bbox_head.cls_convs.0.gn.bias, bbox_head.cls_convs.1.conv.weight, bbox_head.cls_convs.1.gn.weight, bbox_head.cls_convs.1.gn.bias, bbox_head.cls_convs.2.conv.weight, bbox_head.cls_convs.2.gn.weight, bbox_head.cls_convs.2.gn.bias, bbox_head.cls_convs.3.conv.weight, bbox_head.cls_convs.3.gn.weight, bbox_head.cls_convs.3.gn.bias, bbox_head.reg_convs.0.conv.weight, bbox_head.reg_convs.0.gn.weight, bbox_head.reg_convs.0.gn.bias, bbox_head.reg_convs.1.conv.weight, bbox_head.reg_convs.1.gn.weight, bbox_head.reg_convs.1.gn.bias, bbox_head.reg_convs.2.conv.weight, bbox_head.reg_convs.2.gn.weight, bbox_head.reg_convs.2.gn.bias, bbox_head.reg_convs.3.conv.weight, bbox_head.reg_convs.3.gn.weight, bbox_head.reg_convs.3.gn.bias



['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',
 'bfloat16',
 'buffers',
 'children',
 'cpu',
 'cuda',
 'device_ids',
 'dim',
 'double',
 

In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 1.1 task/s, elapsed: 4311s, ETA:     0s

In [6]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.2252308 475.7928 579.7506 554.0585 653.459...,test/0000.jpg
1,0 0.30684522 636.85913 644.42957 688.0863 866....,test/0001.jpg
2,0 0.5021976 881.0813 462.1129 1019.9401 628.43...,test/0002.jpg
3,0 0.095025904 36.20984 154.76361 1004.2351 949...,test/0003.jpg
4,0 0.5378711 419.17603 401.8227 655.4686 578.42...,test/0004.jpg
